In [ ]:
import os
import re
import ast
import pandas as pd
import matplotlib.pyplot as plt
country_iso = ""

if country_iso == "":
    print("Please provide a country iso3")

In [ ]:
from generate_demand_profiles import prepare_appliance_count
prepare_appliance_count(country_iso)


## Collect the simulation output files

In [ ]:
from post_process import collect_profiles_path
sorted_files = collect_profiles_path(folder=f"simulation_data_{country_iso}", output_prefix=f"{country_iso}_all_intermediate_avg")
sorted_files

## Load the AI model data

In [ ]:
df_ai = pd.read_csv(f"{country_iso}_appliance_count.csv")
df_ai

## Merge the simulated profiles in one dataframe

In [ ]:
# Read and merge DataFrames on index
merged_df = pd.DataFrame()

for file_path in sorted_files:
    df = pd.read_csv(file_path, index_col=0)  # use index from file
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = merged_df.join(df, how='outer')  # or 'inner' if strict alignment needed

print(merged_df.head())

## Divide each lga profile by the number of household within the area

In [ ]:
normalized_profiles = merged_df.copy()

In [ ]:
for lga_name in normalized_profiles.columns:
    adm2, adm1 = ast.literal_eval(lga_name)
    normalized_profiles[lga_name] = merged_df[lga_name] / df_ai.loc[(df_ai.shapeName == adm2) & (df_ai.adm1 == adm1), "num_hh"].values[0]

## Figure of all yearly profiles averaged into one day

In [ ]:
ax=normalized_profiles.plot(legend=False, ylabel="Power W",xlabel="Hours of the day",title=f"All RAMP profiles for {country_iso}")
ax.set_xticks([0, 240, 480, (60 * 12), (60 * 16), (60 * 20), (60 * 24)])
ax.set_xticklabels([0, 4, 8, 12, 16, 20, 24])

In [ ]:
fig = ax.get_figure()
fig.savefig(f"All_profiles_{country_iso}.png")

## Figure grouping the profiles per cluster_num

In [ ]:
fig,axes = plt.subplots(1,3,figsize=(14, 7), sharey=True)

colors = ['#56B4E9', '#E69F00', '#009E73']


for cluster_num, color in zip([0,1,2],colors):
    temp = df_ai.loc[df_ai.cluster == cluster_num,["shapeName","adm1"]]

    keys = []
    for adm2, adm1 in zip(temp.shapeName, temp.adm1):
        keys.append(str((adm2, adm1)))
    normalized_profiles[keys].plot(
        ax=axes[cluster_num], 
        legend=False,
        color=color,
        ylabel="Power W",
        title=f"Cluster num={cluster_num}"
    )


for ax in axes:
    ax.set_xticks([0, 240, 480, (60 * 12), (60 * 16), (60 * 20), (60 * 24)])
    ax.set_xticklabels([0, 4, 8, 12, 16, 20, 24])
axes[1].set_xlabel("Hours of the day")




In [ ]:
fig.savefig(f"cluster_comparison_{country_iso}.png")